## Load data: BTC prices 
---


**Bring in BTC price and generate target col for binary classification**

Hence, two labels;

    "1" when BTC Close value rose or remained the same;

    "0" when BTC Close value decreased




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
import seaborn as sns
import requests
import dateutil.parser

from jupyterthemes import jtplot
jtplot.style()

pd.options.display.max_columns = 50  # None -> No Restrictions
pd.options.display.max_rows = 100    # None -> Be careful with this 
pd.options.display.max_colwidth = 400

In [2]:
def get_ticker(ticker, start, end):
    '''
    Input:
        ticker: str, stock ticker (e.g., 'AAPL')
        start: YYYY-MM-DD start date
        end: YYYY-MM-DD end date
    return:
        price dataframe
    '''
    ticker = yf.Ticker(ticker)
    return pd.DataFrame(ticker.history(start=start, end=end))

In [3]:
# Download ticker inputs
start = '2017-12-31'
end = '2020-12-03'
ticker = 'BTC-USD'
BTC = get_ticker(ticker, start, end)
BTC.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-11-29,17719.634766,18283.628906,17559.117188,18177.484375,31133957704,0,0
2020-11-30,18178.322266,19749.263672,18178.322266,19625.835938,47728480399,0,0
2020-12-01,19633.769531,19845.974609,18321.921875,18802.998047,49633658712,0,0
2020-12-02,18801.744141,19308.330078,18347.718750,19201.091797,37387697138,0,0
2020-12-03,19205.925781,19566.191406,18925.785156,19445.398438,31930317405,0,0


In [4]:
# drop cols
cols = ['Dividends','Stock Splits']
BTC.drop(columns=cols,inplace=True)
BTC.head()

,Open,High,Low,Close,Volume
Date,,,,,
2017-12-31,12897.700195,14377.400391,12755.599609,14156.400391,12136299520
2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,10291200000
2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,16846600192
2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,16871900160
2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,21783199744


In [5]:
# add cols
BTC['%'] = BTC['Close'].pct_change()
BTC['vol%_chg1d'] = BTC['Volume'].pct_change()
BTC['vol%_chg3d'] = BTC['Volume'].pct_change(3)
BTC['vol%_chg8d'] = BTC['Volume'].pct_change(8)
BTC['range'] = BTC['High'] / BTC['Low']
BTC['range%'] = (BTC['High'] / BTC['Low']).pct_change()
BTC.dropna(inplace=True)

# generate binary target label
BTC['Target'] = (BTC['%'] > 0).astype(int)
BTC.head(50)

,Open,High,Low,Close,Volume,%,vol%_chg1d,vol%_chg3d,vol%_chg8d,range,range%,Target
Date,,,,,,,,,,,,
2018-01-08,16476.199219,16537.900391,14208.200195,15170.099609,18413899776,-0.079350,0.160589,-0.227634,0.517258,1.163969,0.065188,0
2018-01-09,15123.700195,15497.500000,14424.000000,14595.400391,16659999744,-0.037884,-0.095249,-0.090343,0.618859,1.074425,-0.076930,0
2018-01-10,14588.500000,14973.299805,13691.200195,14973.299805,18500800512,0.025892,0.110492,0.166066,0.098192,1.093644,0.017888,1
2018-01-11,14968.200195,15018.799805,13105.900391,13405.799805,16534099968,-0.104686,-0.106304,-0.102086,-0.020021,1.145957,0.047834,0
2018-01-12,13453.900391,14229.900391,13158.099609,13980.599609,12065699840,0.042877,-0.270254,-0.275768,-0.446101,1.081456,-0.056286,1
2018-01-13,13952.400391,14659.500000,13952.400391,14360.200195,12763599872,0.027152,0.057842,-0.310106,-0.464634,1.050679,-0.028458,1
2018-01-14,14370.799805,14511.799805,13268.000000,13772.000000,11084099584,-0.040960,-0.131585,-0.329622,-0.394794,1.093744,0.040988,0
2018-01-15,13767.299805,14445.500000,13641.700195,13819.799805,12750799872,0.003471,0.150369,0.056781,-0.196344,1.058922,-0.031837,1
2018-01-16,13836.099609,13843.099609,10194.900391,11490.500000,18853799936,-0.168548,0.478637,0.477154,0.023890,1.357845,0.282290,0


In [6]:
# to CSV data folder
BTC.drop(columns=['%','Open','High','Low','Volume'],inplace=True)
BTC.rename(columns={'Date': 'date'},inplace=True)
BTC['Target_1d'] = BTC['Target'].shift(-1)
BTC.to_csv('../thecap/Data/BTC_daily_since_1-2016.csv')

In [7]:
BTC.tail()

,Close,vol%_chg1d,vol%_chg3d,vol%_chg8d,range,range%,Target,Target_1d
Date,,,,,,,,
2020-11-29,18177.484375,-0.045001,-0.492906,-0.214785,1.041261,-0.013752,1,1.0
2020-11-30,19625.835938,0.533004,0.227379,0.156201,1.086418,0.043368,1,0.0
2020-12-01,18802.998047,0.039917,0.522456,0.161263,1.083182,-0.002979,0,1.0
2020-12-02,19201.091797,-0.246727,0.200866,-0.273596,1.052356,-0.028459,1,1.0
2020-12-03,19445.398438,-0.145967,-0.331001,-0.269502,1.033838,-0.017597,1,NaN


In [19]:
# Download Currencys tickers
start = '2017-12-31'
end = '2020-12-03'
ticker = 'JPY=X'
fiat = get_ticker(ticker, start, end)
fiat.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-01-01,112.680000,112.690002,112.669998,112.666000,0,0,0
2018-01-02,112.769997,112.788002,112.074997,112.769997,0,0,0
2018-01-03,112.264000,112.485001,112.193001,112.244003,0,0,0
2018-01-04,112.606003,112.853996,112.507004,112.607002,0,0,0
2018-01-05,112.790001,113.289001,112.731003,112.782997,0,0,0


In [20]:
# drop cols
cols = ['Open','High','Low','Volume','Dividends','Stock Splits']
fiat.drop(columns=cols,inplace=True)
fiat['JPY%'] = fiat['Close'].pct_change()
fiat.dropna(inplace=True)
fiat.rename(columns={'Date': 'date'},inplace=True)
fiat['Close_JPY'] = fiat['Close']
fiat.drop(columns='Close',inplace=True)
fiat.head()

,JPY%,Close_JPY
Date,,
2018-01-02,0.000923,112.769997
2018-01-03,-0.004664,112.244003
2018-01-04,0.003234,112.607002
2018-01-05,0.001563,112.782997
2018-01-08,0.003290,113.153999


In [21]:
# to CSV data folder
fiat.to_csv('../thecap/Data/Jpy_daily_since_12-2017.csv')